<h1 align="center"> LSIR LAB - SEMESTER PROJECT</h1> 

**Students:**
- Etienne BRUNO
- Abiola ADEYE

In [1]:
import pandas as pd
import json

### LOAD DATA

In [2]:
# Import spark and open json file
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.json('../data/sample/sample.jsonl')
json_rdd = df.rdd

df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/10 17:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- __twarc: struct (nullable = true)
 |    |-- retrieved_at: string (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- attachments: struct (nullable = true)
 |    |    |-- media_keys: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- poll_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- author_id: string (nullable = true)
 |    |-- context_annotations: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- domain: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- entity: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- id

In [3]:
spark

In [101]:
json_rdd_en = json_rdd.filter(lambda x: x['includes'] and x['data'] and x['data']['lang'] == 'en')
json_rdd_fr = json_rdd.filter(lambda x: x['includes'] and x['data'] and x['data']['lang'] == 'fr')

In [127]:
from pyspark.sql import Row

en_df = json_rdd_en.map(lambda x: [ x['data']['id'],
                                    x['data']['author_id'],
                                    x['data']['created_at'],
                                    x['data']['public_metrics']['retweet_count'],
                                    x['data']['public_metrics']['reply_count'],
                                    x['data']['public_metrics']['like_count'],
                                    x['data']['public_metrics']['quote_count'],
                                    x['data']['public_metrics']['impression_count'],
                                    x['includes']['users'][0]['public_metrics']['followers_count']]).toDF(['id', 'author_id', 'created_at', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'impression_count', 'followers_count'])


In [129]:
from pyspark.sql import Row

fr_df = json_rdd_fr.map(lambda x: [ x['data']['id'],
                                    x['data']['author_id'],
                                    x['data']['created_at'],
                                    x['data']['public_metrics']['retweet_count'],
                                    x['data']['public_metrics']['reply_count'],
                                    x['data']['public_metrics']['like_count'],
                                    x['data']['public_metrics']['quote_count'],
                                    x['data']['public_metrics']['impression_count'],
                                    x['includes']['users'][0]['public_metrics']['followers_count']]).toDF(['id', 'author_id', 'created_at', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'impression_count', 'followers_count'])


In [ ]:
# save as parquet
en_df.write.parquet('../data/sample/sample_en_parquet/')

In [ ]:
# save as parquet
fr_df.write.parquet('../data/sample/sample_fr_parquet/')

In [8]:
en_df.show(10)

+-------------------+-------------------+--------------------+-------------+-----------+----------+-----------+----------------+----------+-----------+-------------+
|                 id|          author_id|          created_at|retweet_count|reply_count|like_count|quote_count|impression_count|like_count|reply_count|retweet_count|
+-------------------+-------------------+--------------------+-------------+-----------+----------+-----------+----------------+----------+-----------+-------------+
|1633111775397699585|1421990603278061569|2023-03-07T14:26:...|         4644|          0|         0|          0|               0|         0|          0|         4644|
|1633111775406071808|1266246492529913856|2023-03-07T14:26:...|          245|          0|         0|          0|               0|         0|          0|          245|
|1633111775389310976|1516399872173383680|2023-03-07T14:26:...|          121|          0|         0|          0|               0|         0|          0|          121|
|163